In [5]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd

file_path = r"C:\Users\USER\OneDrive\Desktop\Work\amazon_sales_data 2025.csv"
df = pd.read_csv(file_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Order ID           250 non-null    object
 1   Date               250 non-null    object
 2   Product            250 non-null    object
 3   Category           250 non-null    object
 4   Price              250 non-null    int64 
 5   Quantity           250 non-null    int64 
 6   Total Sales        250 non-null    int64 
 7   Customer Name      250 non-null    object
 8   Customer Location  250 non-null    object
 9   Payment Method     250 non-null    object
 10  Status             250 non-null    object
dtypes: int64(3), object(8)
memory usage: 21.6+ KB


In [13]:
!pip install sqlalchemy pymysql pandas


  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install pandas mysql-connector-python


  Using cached mysql_connector_python-9.3.0-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
Using cached mysql_connector_python-9.3.0-cp312-cp312-win_amd64.whl (16.4 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

file_path = r"C:\Users\USER\OneDrive\Desktop\Work\amazon_sales_data 2025.csv"
df = pd.read_csv(file_path)

host = 'localhost'
user = 'root'
password = 'ramyas1507'
database = 'amazondata'

df['order_date'] = pd.to_datetime(df['order_date']).dt.date
df['price'] = df['price'].astype(float)
df['quantity'] = df['quantity'].astype(int)
df['total_sales'] = df['total_sales'].astype(float)


try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    if connection.is_connected():
        cursor = connection.cursor()

       
        cursor.execute("SELECT MAX(id) FROM Orders")
        max_id_result = cursor.fetchone()
        max_id = max_id_result[0] if max_id_result[0] is not None else 0

       
        df.insert(0, 'id', range(max_id + 1, max_id + 1 + len(df)))

       
        insert_query = """
            INSERT INTO Orders (
                id, order_id, order_date, product, category, price, quantity,
                total_sales, customer_name, customer_location, payment_Method, status
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

   
        data_to_insert = [
            tuple(x) for x in df.values
        ]

        cursor.executemany(insert_query, data_to_insert)
        connection.commit()

        print(f"{cursor.rowcount} rows inserted successfully!")

        
        new_max_id = max_id + len(df)
        cursor.execute(f"ALTER TABLE Orders AUTO_INCREMENT = {new_max_id + 1}")
        connection.commit()

        print(f"🔧 AUTO_INCREMENT value updated to {new_max_id + 1}")

except Error as e:
    print(f" Error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("🔌 MySQL connection closed.")


KeyError: 'order_date'

In [25]:
print(df.columns.tolist())


['Order ID', 'Date', 'Product', 'Category', 'Price', 'Quantity', 'Total Sales', 'Customer Name', 'Customer Location', 'Payment Method', 'Status']


In [34]:
import pandas as pd
import mysql.connector

file_path = r"C:\Users\USER\OneDrive\Desktop\Work\amazon_sales_data 2025.csv"
df = pd.read_csv(file_path)

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ramyas1507",
    database="amazondata"
)
cursor = conn.cursor()

cursor.execute("SELECT MAX(id) FROM Orders")
max_id = cursor.fetchone()[0] or 0

df.insert(0, 'id', range(max_id + 1, max_id + 1 + len(df)))

df['order_id'] = df['order_id'].astype(str)

try:
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%y').dt.date
except ValueError:
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.date

df['product'] = df['product'].astype(str)
df['category'] = df['category'].astype(str)
df['price'] = df['price'].astype('float32')
df['quantity'] = df['quantity'].astype(int)
df['total_sales'] = df['total_sales'].astype('float32')
df['customer_name'] = df['customer_name'].astype(str)
df['customer_location'] = df['customer_location'].astype(str)
df['payment_method'] = df['payment_method'].astype(str)
df['status'] = df['status'].astype(str)

data = [
    (
        row.order_id, row.date, row.product, row.category, row.price,
        row.quantity, row.total_sales, row.customer_name,
        row.customer_location, row.payment_method, row.status
    )
    for row in df.itertuples(index=False)
]

sql = """
INSERT INTO Orders (
    order_id, order_date, product, category, price, quantity, total_sales, 
    customer_name, customer_location, payment_Method, status
) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    cursor.executemany(sql, data)
    conn.commit()
    print(f"{cursor.rowcount} records inserted successfully.")

    # new_max_id = max_id + len(df)
    # cursor.execute(f"ALTER TABLE Orders AUTO_INCREMENT = {new_max_id + 1}")
    # conn.commit()
    # print(f" AUTO_INCREMENT set to {new_max_id + 1}")

except Exception as e:
    print(f" Error during insert: {e}")
    conn.rollback()

cursor.close()
conn.close()




250 records inserted successfully.
